In [ ]:

import datetime
import time

import pandas as pd
from numpy import linalg as la

from sklearn.ensemble import RandomForestClassifier
from sklearn import svm

import numpy as np
import random

def predict(startime,endtime):
    train = dataset(startime,endtime)
    train=dealdata(train)
    delat2= datetime.timedelta(days=2)
    #print('2',endtime-delat2,endtime)
    test = dataset(endtime-delat2,endtime)
   # print (endtime-delat2,endtime)
    test=dealdata(test)
    col = [c for c in train.columns if c not in ['Return','Unnamed: 0','Y_bin']]

    rfc=RandomForestClassifier()
    rfc.fit(train[col],train['Y_bin'])
    t1=rfc.predict(test[col])#test[col]为交易日前两天的股票基本面信息，t1为预测的结果涨幅能在前百分之几的股票列表，顺序没变
    return t1,test
    
def dealdata(data):
#     data = data.drop(['capital_reserve_fund','operating_revenue','gross_profit_margin','inc_total_revenue_year_on_year'],axis=1)
    col = [c for c in data.columns if c not in ['Return','Unnamed: 0','Y_bin']]
    
    for i in col:
        data[i].fillna(data[i].mean(),inplace=True)
    data['Y_bin'].fillna(-1,inplace=True)
    data['Return'].fillna(-1,inplace=True)
    amin, amax = data.min(), data.max() # 求最大最小值
    data = (data-amin)/(amax-amin) # (矩阵元素-最小值)/(最大值-最小值)
    return data
    
                    
def dataset(startdate,enddate,window,type,stockindex):
    
    fdate=startdate
    stock_set = get_industry_stocks(stockindex,fdate)#得到fdate当天的stockind按行业分类股票池中的股票代码
#     stock_set=get_index_stocks(stockindex,fdate)
    q = query(
        valuation.code,
        indicator.roa,
        valuation.ps_ratio,
        valuation.pcf_ratio,
        indicator.roe,
        indicator.gross_profit_margin,
        
        
        valuation.circulating_market_cap,
        valuation.pb_ratio,
        valuation.pcf_ratio,
        balance.capital_reserve_fund,
        income.operating_revenue,
        indicator.inc_total_revenue_year_on_year,
        indicator.roe,
        indicator.gross_profit_margin,
        indicator.operation_profit_to_total_revenue,
        indicator.inc_revenue_year_on_year,
        indicator.inc_revenue_annual
        ).filter(
        valuation.code.in_(stock_set)
    )
    fdf = get_fundamentals(q, date=fdate)
    #获取股票代码中的各个指标
    fdf.index = fdf['code']
    fdf.pop('code');

    current_date=startdate
    forcast_date=enddate
    current_close=get_price(stock_set, fields=['close'],end_date=current_date, count=1)['close'].T#获取股票列表中的股票在所设置时间区间的收盘价
    forcast_close=get_price(stock_set, fields=['close'],end_date=forcast_date, count=1)['close'].T

    grow=(forcast_close.iloc[:,0]-current_close.iloc[:,0])/current_close.iloc[:,0]#求涨幅
    grow=pd.DataFrame(grow, columns=['Return'])
    
    df =pd.merge(fdf,grow,left_index =True, right_index=True)#拼接
    if type==1:#选择第一种阈值计算类型（前百分之window的涨幅为1类，之后的为0类）
        bound = np.nanpercentile(df['Return'],window)
        df.loc[(df['Return']>=bound),'Y_bin']=1
        df.loc[(df['Return']<bound),'Y_bin']=-1
    else:#涨的为1类，跌的为0类（用于短期阈值比较好）
        bound = 0
        df.loc[(df['Return']>=bound),'Y_bin']=1
        df.loc[(df['Return']<bound),'Y_bin']=-1
    
    return df

def dataset_for_index(startdate,enddate,window,type,stockindex):

    fdate=startdate
#     stock_set = get_industry_stocks(stockindex,fdate)#得到fdate当天的stockind按行业分类股票池中的股票代码
    stock_set=get_index_stocks(stockindex,fdate)
    q = query(
        valuation.code,
        indicator.roa,
        valuation.ps_ratio,
        valuation.pcf_ratio,
        indicator.roe,
        indicator.gross_profit_margin,
        
        
        valuation.circulating_market_cap,
        valuation.pb_ratio,
        valuation.pcf_ratio,
        balance.capital_reserve_fund,
        income.operating_revenue,
        indicator.inc_total_revenue_year_on_year,
        indicator.roe,
        indicator.gross_profit_margin,
        indicator.operation_profit_to_total_revenue,
        indicator.inc_revenue_year_on_year,
        indicator.inc_revenue_annual
        ).filter(
        valuation.code.in_(stock_set)
    )
    fdf = get_fundamentals(q, date=fdate)
    #获取股票代码中的各个指标
    fdf.index = fdf['code']
    fdf.pop('code');

    current_date=startdate
    forcast_date=enddate
    current_close=get_price(stock_set, fields=['close'],end_date=current_date, count=1)['close'].T#获取股票列表中的股票在所设置时间区间的收盘价
    forcast_close=get_price(stock_set, fields=['close'],end_date=forcast_date, count=1)['close'].T

    grow=(forcast_close.iloc[:,0]-current_close.iloc[:,0])/current_close.iloc[:,0]#求涨幅
    grow=pd.DataFrame(grow, columns=['Return'])
    
    df =pd.merge(fdf,grow,left_index =True, right_index=True)#拼接
    if type==1:#选择第一种阈值计算类型（前百分之window的涨幅为1类，之后的为0类）
        bound = np.nanpercentile(df['Return'],window)
        df.loc[(df['Return']>=bound),'Y_bin']=1
        df.loc[(df['Return']<bound),'Y_bin']=-1
    else:#涨的为1类，跌的为0类（用于短期阈值比较好）
        bound = 0
        df.loc[(df['Return']>=bound),'Y_bin']=1
        df.loc[(df['Return']<bound),'Y_bin']=-1
    
    return df



def ROC(prediction,test,trueclass=1,flaseclass=0):
    '''
    :param prediction: 预测标志
    :param test: 真实标志
    :param trueclass: 视为真的类别值
    :param flaseclass: 视为假的类别值
    :return:
    '''
    #这里写的字母表示意义与教材的不一致，因此后面计算真阳率时不能用书上的公式
    TP=0#预测真为真
    FN=0#预测真为假
    FP=0#预测假为真
    TN=0#预测假为假

    for i in range(np.shape(test)[0]):
        
        if prediction[i]==trueclass and test[i]==trueclass:
            TP=TP+1
        elif prediction[i]==flaseclass and test[i]==flaseclass:
            TN=TN+1
        elif prediction[i]==trueclass and test[i]==flaseclass:
            FP=FP+1
        elif prediction[i]==flaseclass and test[i]==trueclass:
            FN=FN+1

    if (FP+TN)==0:#因为样本选取可能没有假例，此时预测假为假的TN值永远为零，当FP也为零时，分母为零，报错，这样做为了防止报错，因此最好不要用只有一类的样本集
        return TP/(TP+FN),0
    return TP/(TP+FN),FP / (FP + TN)

def print_(random_seed,s,train_data):
    rfc=RandomForestClassifier(n_estimators=10, max_depth=None,min_samples_split=2, random_state=random_seed)
    rfc.fit(train_data[col],train_data['Y_bin'])
    importances_result_dict={}
    importance=rfc.feature_importances_
    for i in range(len(col)):
        importances_result_dict[col[i]]=importance[i]
    result=sorted(importances_result_dict.items(),key=lambda item:item[1],reverse=True)
    im=[]
    score=[]
    for i in range(len(col)):
        im.append(result[i][0])
        score.append(result[i][1])
    plt.subplot(s)
    plt.xticks(list(map(lambda x,y:x+y,list(range(len(col))),[0.5]*len(col))),im,rotation=90)
    plt.bar(range(len(col)),score)

def train_max_index(train_startday,train_endday,test_startday,test_endday,stockindex,train_data,test_data):
    x=[]
    y=[]
    
    max_index=0#存储最大真阳率的random_state值
    max_zhenyang=0
    for i in range(100):
        rfc=RandomForestClassifier(n_estimators=10, max_depth=None,min_samples_split=2, random_state=i)
        rfc.fit(train_data[col],train_data['Y_bin'])
        t1=rfc.predict(test_data[col])

        zhenyang,jiayang=ROC(t1,np.array(test_data['Y_bin']))
        if max_zhenyang<zhenyang:
            max_index=i
            max_zhenyang=zhenyang
    return max_index
    

利用价值： 例如已知8月到10月的数据 那么通过下列方法选出因子可以解释为8月到10月上涨的主要参考因子是排名前面的几个，那么考虑到趋势是不会突然变向的，就有理由去相信下个月上涨主要参考因子还是前面这几个，因此可以对下个月的选股做一个参考。 所以，可以这么认为： 对于小时间段而言，跑出来的因子就是对于这个小段时间内有效的 那么如果把时间段放长，那么得到的因子就是中长线因子

In [ ]:
import matplotlib.pyplot as plt
# stockindex='HY008'
stockindex='000001.XSHG'
train_startday='2017-08-02'
train_endday='2017-09-02'
test_startday='2017-09-02'
test_endday='2017-10-02'
train_data= dealdata(dataset_for_index(train_startday,train_endday,70,1,stockindex))
test_data=dealdata(dataset_for_index(test_startday,test_endday,70,1,stockindex))

col = [c for c in train_data.columns if c not in ['Return','Unnamed: 0','Y_bin']]
max_index=train_max_index(train_startday,train_endday,test_startday,test_endday,stockindex,train_data,test_data)  
#使用可以得到最优真阳率的随机种子来训练，然后看模型的特征重要性得分，会发现每股营收和市值排名最前，也正好验证了这段时间涨的都是大盘蓝筹股的事实
plt.title(train_startday+' '+test_endday+' '+stockindex)
print_(max_index,111,train_data)
plt.show()

In [ ]:
import matplotlib.pyplot as plt
# stockindex='HY008'
stockindex='000001.XSHG'
train_startday='2015-08-02'
train_endday='2016-08-02'
test_startday='2016-08-02'
test_endday='2017-08-02'
train_data= dealdata(dataset_for_index(train_startday,train_endday,70,1,stockindex))
test_data=dealdata(dataset_for_index(test_startday,test_endday,70,1,stockindex))

col = [c for c in train_data.columns if c not in ['Return','Unnamed: 0','Y_bin']]
max_index=train_max_index(train_startday,train_endday,test_startday,test_endday,stockindex,train_data,test_data)  
#使用可以得到最优真阳率的随机种子来训练，然后看模型的特征重要性得分，会发现每股营收和市值排名最前，也正好验证了这段时间涨的都是大盘蓝筹股的事实
plt.title(train_startday+' '+test_endday+' '+stockindex)
print_(max_index,111,train_data)
plt.show()

In [ ]:
#通过真阳率选出的最优模型与随机的模型进行比较（变量为随机种子）
import random
plt.title(str(max_index))
print_(max_index,111,train_data)
plt.show()
for i in range(5):
    plt.title('random')
    print_(int(random.uniform(1,100)),111,train_data)
    plt.show()

可以发现，贴合实际最准确的还是用最优模型选出来的股
接下来要以板块为变量了，上面用的是上证指数 个人认为用上证指数跑出来的结果准确率没有那么高，因为A股涉及到题材炒作，同一个板块的股票走势往往都会互相影响，所以直接用每个行业作为股票池的话，就相当于给原来的上证指数训练集降维了，话不多说，来看看结果。

In [ ]:
stockindex='HY008'
train_data= dealdata(dataset(train_startday,train_endday,70,1,stockindex))
test_data=dealdata(dataset(test_startday,test_endday,70,1,stockindex))

col = [c for c in train_data.columns if c not in ['Return','Unnamed: 0','Y_bin']]
max_index=train_max_index(train_startday,train_endday,test_startday,test_endday,stockindex,train_data,test_data)  
#使用可以得到最优真阳率的随机种子来训练，然后看模型的特征重要性得分，会发现每股营收和市值排名最前，也正好验证了这段时间涨的都是大盘蓝筹股的事实
plt.title(train_startday+' '+test_endday+' '+stockindex)
print_(max_index,111,train_data)
plt.show()

果然，市值的区分度变得更加明显了，那么别的行业呢

In [ ]:
for i in range(1,10):
    stockindex='HY00'+str(i)
    train_data= dealdata(dataset(train_startday,train_endday,70,1,stockindex))
    test_data=dealdata(dataset(test_startday,test_endday,70,1,stockindex))

    col = [c for c in train_data.columns if c not in ['Return','Unnamed: 0','Y_bin']]
    max_index=train_max_index(train_startday,train_endday,test_startday,test_endday,stockindex,train_data,test_data)  
    #使用可以得到最优真阳率的随机种子来训练，然后看模型的特征重要性得分，会发现每股营收和市值排名最前，也正好验证了这段时间涨的都是大盘蓝筹股的事实
    plt.title(train_startday+' '+test_endday+' '+stockindex)
    print_(max_index,111,train_data)
    plt.show()

In [ ]:
通过这个办法是不是可以知道每个行业上涨主要依据是什么了呢
逻辑： 先是以上证指数作为训练集，经过一系列运算后得到走势依据的结果排名，发现排名结果与实际情况符合，随即猜想这个随机森林模型的这种用法是有效的。那么在假设这个模型是有效的前提下，当训练集变成各个板块的时候，所反映出来的结果也是有效的。
结论： 在进行多因子选股的时候，每个板块主要依据的因子是不同的，可以用以上的方法对每个板块的每个因子分配与其重要性得分相对应的权重。